<a href="https://colab.research.google.com/github/Daramluv/Google_colab/blob/main/Employee_Scheduling_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 33.1 MB/s eta 0:00:00


In [2]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
# compute number of cominations
def nCr(n, r):
    return int(factorial(n) / (factorial(r) * factorial(n - r)))

# compute availability table
# w is the length of the shift, d is number of days off
def schedule(w,d):

    # number of shifts
    v = nCr(w,d)
    s = [[1 for x in range(w)] for x in range(v)]

    i=0
    for j in range(v):
        for k in range(j+1,w):
            s[i][j] = 0
            s[i][k] = 0
            i=i+1
    return s

In [4]:
# generate the schedule availability table
a = schedule(7,2)

# print the table to visualize
print(np.matrix(a))

[[0 0 1 1 1 1 1]
 [0 1 0 1 1 1 1]
 [0 1 1 0 1 1 1]
 [0 1 1 1 0 1 1]
 [0 1 1 1 1 0 1]
 [0 1 1 1 1 1 0]
 [1 0 0 1 1 1 1]
 [1 0 1 0 1 1 1]
 [1 0 1 1 0 1 1]
 [1 0 1 1 1 0 1]
 [1 0 1 1 1 1 0]
 [1 1 0 0 1 1 1]
 [1 1 0 1 0 1 1]
 [1 1 0 1 1 0 1]
 [1 1 0 1 1 1 0]
 [1 1 1 0 0 1 1]
 [1 1 1 0 1 0 1]
 [1 1 1 0 1 1 0]
 [1 1 1 1 0 0 1]
 [1 1 1 1 0 1 0]
 [1 1 1 1 1 0 0]]


In [5]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('employee scheduling')

shift = [*range(0,21)]
day = [*range(0,7)]

p = [200,200,200,200,200,300,300]

d=[177,134,158,191,149,165,116]

# Cartesian product of shifts and days
sd = []

for i in shift:
    for j in day:
        tp = i,j
        sd.append(tp)

#print(np.matrix(sd))

# Build decision variables: how many employees to work on a shift
x = m.addVars(shift, vtype=GRB.INTEGER, name='Assign')

# Objective function: Minimize total payroll cost
m.setObjective(gp.quicksum(p[j]*a[i][j]*x[i] for i,j in sd), GRB.MINIMIZE)

# Satisfying staffing requirements
staffingConstrs = m.addConstrs((gp.quicksum(x[i]*a[i][j]  for i in shift) >= d[j] for j in day),
                                      name='staffingConstrs')

# Run optimization engine
m.optimize()

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 21 columns and 105 nonzeros
Model fingerprint: 0x8978f0cd
Variable types: 0 continuous, 21 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Found heuristic solution: objective 458100.00000
Presolve time: 0.00s
Presolved: 7 rows, 21 columns, 105 nonzeros
Variable types: 0 continuous, 21 integer (0 binary)

Root relaxation: objective 2.461000e+05, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/N

In [6]:
#####################################################
#         Number of employees on each shift
#####################################################

print(f"\n\n___Optimal employees on each shift________")
total_employee = 0
for t in shift:
    print("The number of employee on shift %2d is %3d" % (t, x[t].x))
    #print(f"The number of employee on shift {t} is {x[t].x}")
    total_employee += x[t].x

print("The total number of emploee is %3d " % (total_employee))



___Optimal employees on each shift________
The number of employee on shift  0 is   0
The number of employee on shift  1 is   0
The number of employee on shift  2 is  10
The number of employee on shift  3 is   0
The number of employee on shift  4 is   0
The number of employee on shift  5 is  31
The number of employee on shift  6 is  37
The number of employee on shift  7 is   0
The number of employee on shift  8 is  46
The number of employee on shift  9 is   0
The number of employee on shift 10 is   1
The number of employee on shift 11 is   0
The number of employee on shift 12 is  23
The number of employee on shift 13 is   0
The number of employee on shift 14 is   0
The number of employee on shift 15 is   0
The number of employee on shift 16 is   0
The number of employee on shift 17 is  17
The number of employee on shift 18 is   0
The number of employee on shift 19 is   0
The number of employee on shift 20 is  53
The total number of emploee is 218 


In [7]:
#####################################################
#     Actual coverage on each day
#####################################################

print(f"\n\n____Actual coverage on each day_____")
for t in day:
    actual_employee = 0
    for s in shift:
      actual_employee += (x[s].x)*a[s][t]
    print("The number of employee on shift %1d is %3d where demand is %3d" % (t, actual_employee, d[t]) )



____Actual coverage on each day_____
The number of employee on shift 0 is 177 where demand is 177
The number of employee on shift 1 is 134 where demand is 134
The number of employee on shift 2 is 158 where demand is 158
The number of employee on shift 3 is 191 where demand is 191
The number of employee on shift 4 is 149 where demand is 149
The number of employee on shift 5 is 165 where demand is 165
The number of employee on shift 6 is 116 where demand is 116


In [8]:
# assuming employees must take two consequtive days off
# compute availability table
# w is the length of the shift, d is number of days off
def schedule(w,d):

    s = [[1 for x in range(w)] for x in range(w)]
    i=0
    for j in range(w):
        s[i][j] = 0
        s[i][(j+1)%w] = 0
        i=i+1
    return s

In [9]:
# generate the schedule availability table
a = schedule(7,2)

# print the table to visualize
print(np.matrix(a))

[[0 0 1 1 1 1 1]
 [1 0 0 1 1 1 1]
 [1 1 0 0 1 1 1]
 [1 1 1 0 0 1 1]
 [1 1 1 1 0 0 1]
 [1 1 1 1 1 0 0]
 [0 1 1 1 1 1 0]]


In [10]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('employee scheduling')

shift = [*range(0,7)]
day = [*range(0,7)]

p = [200,200,200,200,200,300,300]

d=[177,134,158,191,149,165,116]

# Cartesian product of shifts and days
sd = []

for i in shift:
    for j in day:
        tp = i,j
        sd.append(tp)

#print(np.matrix(sd))

# Build decision variables: how many employees to work on a shift
x = m.addVars(shift, vtype=GRB.INTEGER, name='Assign')

# Objective function: Minimize total payroll cost
m.setObjective(gp.quicksum(p[j]*a[i][j]*x[i] for i,j in sd), GRB.MINIMIZE)

# Satisfying staffing requirements
staffingConstrs = m.addConstrs((gp.quicksum(x[i]*a[i][j]  for i in shift) >= d[j] for j in day),
                                      name='staffingConstrs')

# Run optimization engine
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 7 columns and 35 nonzeros
Model fingerprint: 0x705c5c9a
Variable types: 0 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Found heuristic solution: objective 370100.00000
Presolve time: 0.00s
Presolved: 7 rows, 7 columns, 35 nonzeros
Variable types: 0 continuous, 7 integer (0 binary)

Root relaxation: objective 2.584333e+05, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 258433.333    0    4 370100.000 258433.333  30.2%    

In [11]:
#####################################################
#         Number of employees on each shift
#####################################################

print(f"\n\n___Optimal employees on each shift________")
total_employee = 0
for t in shift:
    print("The number of employee on shift %2d is %3d" % (t, x[t].x))
    #print(f"The number of employee on shift {t} is {x[t].x}")
    total_employee += x[t].x

print("The total number of emploee is %3d " % (total_employee))



___Optimal employees on each shift________
The number of employee on shift  0 is   4
The number of employee on shift  1 is  73
The number of employee on shift  2 is   0
The number of employee on shift  3 is  39
The number of employee on shift  4 is   0
The number of employee on shift  5 is  66
The number of employee on shift  6 is  49
The total number of emploee is 231 


In [12]:
#####################################################
#     Actual coverage on each day
#####################################################

print(f"\n\n____Actual coverage on each day_____")
for t in day:
    actual_employee = 0
    for s in shift:
      actual_employee += (x[s].x)*a[s][t]
    print("The number of employee on shift %1d is %3d where demand is %3d" % (t, actual_employee, d[t]) )



____Actual coverage on each day_____
The number of employee on shift 0 is 178 where demand is 177
The number of employee on shift 1 is 154 where demand is 134
The number of employee on shift 2 is 158 where demand is 158
The number of employee on shift 3 is 192 where demand is 191
The number of employee on shift 4 is 192 where demand is 149
The number of employee on shift 5 is 165 where demand is 165
The number of employee on shift 6 is 116 where demand is 116


---
##  Conclusion

In this example, we addressed the employee scheduling problem. We determined the optimal number of employee to:
* Satisfy demand for each day,
* Minimize the total number of employees, and
* Find number of employees to place in each shift.

We also consider variations of the model where employees must take two conseutive days off.

Our employee scheduling model can be used by many organizations to help make informed decisions about which shifts and how many employees to have in order to satisfy daily demand while minimizing the number of employees on the payroll.